In [26]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def WebScrape(Driver, Business, Location):
    
    final_collection = [] # List to collect all reviews
    
    driver = webdriver.Chrome(Driver)
    driver.get("https://www.google.com")
    
    search_box = driver.find_element(By.NAME, "q") # Searching for the business name and location on Google
    search_box.send_keys(f"{Business} {Location} reviews")
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(1)  
    
    # Waiting for the review button to be clickable
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-async-trigger='reviewDialog']"))
    )
    
    # getting the number of reviews
    review_text = review_button.text
    review_counts_str = review_text.split(" Google reviews")[0].replace(",", "")
    review_counts = int(review_counts_str)
    
    # Click the reviews button to open the review section
    review_button.click()

    review_limit = min(review_counts, 500)  # Set review limit to 500 or less if fewer reviews are available
    review_count = 0

    # Wait for the review dialog to load
    review_dialog = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "review-dialog-list"))
    )
    
    # Scroll and scrape reviews
    while review_count < review_limit:
        # Use BeautifulSoup to parse the current page content
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find all reviews (adjust class as necessary)
        reviews = soup.find_all("span", class_="review-full-text")  # Modify the class if needed

        # Update the review count
        review_count += len(reviews)
        print(f"Total reviews collected: {review_count}")

        # Extract and append reviews
        for review in reviews:
            final_collection.append(review.get_text())
        
        # If review limit is reached, stop scraping
        if review_count >= review_limit:
            print(f"Collected {review_count} reviews. Stopping scraping.")
            break
        
        # Scroll within the review dialog to load more reviews
        scrollable_element = driver.find_element(By.CLASS_NAME, "review-dialog-list")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
        time.sleep(1)  # Adjust delay based on page load speed
    
    # Close the driver after scraping is done
    driver.quit()

    # Print the time taken for scraping
    b = time.time()
    print(f"Time taken: {b - a} seconds")
    
    # Return the collected reviews
    return final_collection

# Example usage
Driver = r'.\chromedriver-win64\chromedriver.exe'
Business = "Dominos Pizza"
Location = "Yaba"

collection = WebScrape(Driver, Business, Location)


Total reviews collected: 6
Total reviews collected: 14
Total reviews collected: 28
Total reviews collected: 43
Total reviews collected: 58
Total reviews collected: 76
Total reviews collected: 97
Total reviews collected: 123
Total reviews collected: 154
Total reviews collected: 190
Total reviews collected: 229
Total reviews collected: 272
Total reviews collected: 317
Total reviews collected: 363
Total reviews collected: 412
Total reviews collected: 466
Total reviews collected: 521
Collected 521 reviews. Stopping scraping.
Time taken: 77.47576093673706 seconds


521